In [1]:
from IPython import utils  
from IPython.core.display import HTML  
import os  
def css_styling():  
    """Load default custom.css file from ipython profile"""
    base = utils.path.get_ipython_dir()
    styles = "<style>\n%s\n</style>" % (open(os.path.join(base,'profile_default/static/custom/custom.css'),'r').read())
    return HTML(styles)
css_styling()  

/Users/bolsen/.virtualenvs/urwobject/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: get_ipython_dir has moved to the IPython.paths module since IPython 4.0.
  


In [2]:
import os
import pandas as pd
import numpy as np
import struct
from IPython.display import display, HTML
from collections import namedtuple
from enum import Enum


OBJ_BYTES = 172
GAME_RESOURCES = "/Users/bolsen/Desktop/URW/steam_urw/UrW.app/Contents/Resources/"

In [3]:
def iterate_object_bytes(filename):
    with open(os.path.join(filename), 'rb') as infile:
        data = infile.read()
    assert len(data) % OBJ_BYTES == 0
    for i in range(len(data) // OBJ_BYTES):
        yield data[i * OBJ_BYTES: (i+1) * OBJ_BYTES]

In [4]:
class ObjectType(Enum):
    unused = 0
    weapon = 1
    armor = 2
    container = 3
    map = 4
    skin= 5
    tool = 6
    food = 7
    timber = 8
    watercraft = 9
    valuable = 10
    plant = 11
    carcass = 12

In [5]:
def parse_byte_string(b, method='truncate', null_string='tuntematon'):
    if method == 'mask':
        # Returns all the bytes, with null bytes masked out
        out = ''.join(chr(c) if c != 0 else '.' for c in b)
    elif method == 'truncate':
        # Returns the null-terminated string
        out = []
        for c in b:
            if c == 0:
                break
            out.append(chr(c))
        out = ''.join(out)
    if out == null_string:
        return None
    return out

def parse_float(b):
    assert len(b) == 4
    return struct.unpack('f', b)[0]

In [6]:
class ByteData(object):
    def __init__(self):
        pass

# 172 total bytes of data, this defines where all the data is in that structure
urw_object_structure = {}

urw_object_structure['object_type'] = ByteData(0, 'type_lookup')
urw_object_structure['item_name'] = ByteData((9,49), 'string')
urw_object_structure['item_group'] = ByteData((49,76), 'string', null_string='tuntematon')
urw_object_structure['sprite_name'] = ByteData((76,89), 'string')
urw_object_structure['sprite_index'] = ByteData(90, 'int')
urw_object_structure['value'] = ByteData((92,96), 'float')
urw_object_structure['blunt'] = ByteData(96, 'int', req_type=['weapon', 'armor'])
urw_object_structure['edge'] = ByteData(97, 'int', req_type=['weapon', 'armor'])
urw_object_structure['point'] = ByteData(98, 'int', req_type=['weapon', 'armor'])
urw_object_structure['tear'] = ByteData(99, 'int', req_type=['weapon', 'armor'])
urw_object_structure['squeeze'] = ByteData(100, 'int', req_type=['weapon', 'armor'])
urw_object_structure['warmth'] = ByteData(101, 'int', req_type=['weapon', 'armor'])
urw_object_structure['carb'] = ByteData(96, 'int', req_type=['food', 'plant'])
urw_object_structure['fat'] = ByteData(97, 'int', req_type=['food', 'plant'])
urw_object_structure['protein'] = ByteData(98, 'int', req_type=['food', 'plant'])
urw_object_structure['mature'] = ByteData(99, 'int', req_type=['food', 'plant'])
urw_object_structure['sprout'] = ByteData(100, 'int', req_type=['food', 'plant'])
urw_object_structure['wither'] = ByteData(101, 'int', req_type=['food', 'plant'])
urw_object_structure['weight1'] = ByteData((112,116), 'float')
urw_object_structure['weight2'] = ByteData((116,120), 'float')
urw_object_structure['water'] = ByteData(148, 'int')
urw_object_structure['valueperlbs'] = ByteData((152,156), 'float')
urw_object_structure['quality'] = ByteData(165, 'int')
    
# Byte locations that are still unidentified (0-based):
"""
1-8 inclusive
89
91
102-111
120-147
149-151
156-164
166-171
"""

TypeError: __init__() takes 1 positional argument but 3 were given

In [7]:
def parse_object(raw_bytes):
    unk = []
    unk.append(raw_bytes[1:9]) # often zero, not for plants/food/others? # not floats
    unk.append(raw_bytes[89]) # often zero, not always?
    unk.append(raw_bytes[91]) # always zero
    unk.append(raw_bytes[102:112])
    unk.append(raw_bytes[120:148])
    unk.append(raw_bytes[149:152]) # always zero
    unk.append(raw_bytes[156:165]) # always zero
    unk.append(raw_bytes[166:172])
    
    adj_unk = []
    for u in unk:
        if type(u) == bytes:
            u = ''.join(format(x, '02x') for x in u)
            if all([c == '0' for c in u]):
                u = None
        else:
            if u == 0:
                u = None
        adj_unk.append(u)
    unk = adj_unk
            
    obj_type = ObjectType(raw_bytes[0]).name
    name = parse_byte_string(raw_bytes[9:49], 'truncate')
    group = parse_byte_string(raw_bytes[49:76], 'truncate')
    if group == 'tuntematon':
        group = None
    sprite_name = parse_byte_string(raw_bytes[76:89], 'truncate')
    sprite_index = raw_bytes[90]
    value = struct.unpack('f', raw_bytes[92:96])[0]
    next_six = [b for b in raw_bytes[96:102]]
    blunt, edge, point, tear, squeeze, warmth = None, None, None, None, None, None
    carb, fat, protein, mature, sprout, wither = None, None, None, None, None, None
    if obj_type in ['weapon', 'armor']:
        blunt, edge, point, tear, squeeze, warmth = next_six
    elif obj_type in ['food', 'plant']:
        carb, fat, protein, mature, sprout, wither = next_six
    
    weight1 = struct.unpack('f', raw_bytes[112:116])[0]
    weight2 = struct.unpack('f', raw_bytes[116:120])[0]
    water = raw_bytes[148] # Water content of berries
    valueperlbs = struct.unpack('f', raw_bytes[152:156])[0]
    quality = raw_bytes[165]
    
    
    
    return [obj_type, name, group, sprite_name, sprite_index, value,
            blunt, edge, point, tear, squeeze, warmth,
            carb, fat, protein, mature, sprout, wither,
            weight1, weight2, water, valueperlbs, quality] + unk

In [9]:
objects = [parse_object(o) for o in iterate_object_bytes(os.path.join(GAME_RESOURCES, 'CONSTANT.OBJ'))]
#objects = [parse_object(o) for o in iterate_object_bytes(os.path.join(GAME_RESOURCES, 'TEST/TEST.OBJ'))]
objects = [obj for obj in objects if obj[0] != 'unused']

cols = ['Type', 'Name', 'Group', 'Sprite Name', 'Sprite Index', 'Value',
        'Blunt', 'Edge', 'Point', 'Tear', 'Squeeze', 'Warmth',
        'Carb', 'Fat', 'Protein', 'Mature', 'Sprout', 'Wither',
        'Weight1', 'Weight2', 'Water', 'ValuePerLbs', 'Quality', 
        'UK1', 'UK2', 'UK3', 'UK4', 'UK5', 'UK6', 'UK7', 'UK8']
df = pd.DataFrame(data=sorted(objects, key=lambda x:(x[0], x[1])), columns=cols)

In [10]:
cols = ['Type', 'Name', 'Group', 'Sprite Name', 'Value',
        'Carb', 'Fat', 'Protein',
        'Weight1', 'Weight2', 'Water', 'Quality']

mask = np.array(['club' in name.lower() for name in df['Name']])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[mask][cols])

,Type,Name,Group,Sprite Name,Value,Carb,Fat,Protein,Weight1,Weight2,Water,Quality
283,weapon,Club,CLUB,wp-club,2.0,NaN,NaN,NaN,3.0,3.0,0,0


In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,Type,Name,Group,Sprite Name,Sprite Index,Value,Blunt,Edge,Point,Tear,Squeeze,Warmth,Carb,Fat,Protein,Mature,Sprout,Wither,Weight1,Weight2,Water,ValuePerLbs,Quality,UK1,UK2,UK3,UK4,UK5,UK6,UK7,UK8
0,armor,Bandage,None,it-bandage,97,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0,0.00,0,None,NaN,None,00000000000000030000,0000000000000000000000000000000000000000200000...,None,None,None
1,armor,Birch-bark cap,None,ar-bbcap,82,8.00,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.900000,0.900000,0,0.00,0,None,NaN,None,00000000010000000000,None,None,None,None
2,armor,Birch-bark necklace,None,ar-bbneckl,82,8.00,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.100000,0,0.00,0,None,NaN,None,00000000080000000000,None,None,None,None
3,armor,Birch-bark shoes,None,ar-bbshoes,83,16.00,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.800000,0.800000,0,0.00,0,None,NaN,None,00000000000001000000,None,None,None,None
4,armor,Fur cap,None,ar-furcap,82,24.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0,0.00,0,None,NaN,None,00000000010000000000,None,None,None,None
5,armor,Fur cloak,None,ar-furcloak,80,240.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,11.500000,11.500000,0,0.00,0,None,NaN,None,00000000107e00000000,None,None,None,None
6,armor,Fur footwear,None,ar-furboots,103,96.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.800000,2.800000,0,0.00,0,None,NaN,None,00000000008001000000,None,None,None,None
7,armor,Fur hood,None,ar-furhood,84,32.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.076923,1.076923,0,0.00,0,None,NaN,None,000000000d0000000000,None,None,None,None
8,armor,Fur leggings,None,ar-furlegg,103,104.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,7.420000,7.420000,0,0.00,0,None,NaN,None,0000000000f800000000,None,None,None,None
9,armor,Fur mittens,None,ar-mittens,84,24.00,5.0,3.0,2.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0,0.00,0,None,NaN,None,00000000800100000000,None,None,None,None
